# test
https://github.com/SpencerPark/IJava

%maven com.typesafe.akka:akka-actor-typed_2.13:2.6.8
List<String> added = %jars /home/guci/.m2/repository/com/typesafe/akka/akka-actor_2.11/2.4.20/*.jar

In [1]:
%%loadFromPOM
<dependency>
    <groupId>com.typesafe.akka</groupId>
    <artifactId>akka-actor-typed_2.13</artifactId>
    <version>2.6.8</version>
</dependency>
<dependency>
    <groupId>ch.qos.logback</groupId>
    <artifactId>logback-classic</artifactId>
    <version>1.2.3</version>
</dependency>


In [2]:
import java.util.HashMap;
HashMap<String, String> capitalCities = new HashMap<String, String>();

capitalCities.put("df","ff");
String s=capitalCities.get("df");
System.out.println(s)

ff


# akka
https://developer.lightbend.com/start/?group=akka

In [3]:
import akka.actor.typed.ActorRef;
import akka.actor.typed.Behavior;
import akka.actor.typed.javadsl.*;
import akka.actor.typed.ActorSystem;
import java.io.IOException;

In [18]:
public class TestActor extends AbstractBehavior<TestActor.HelloMsg> {

    public static class HelloMsg {
        public final String name;

        public HelloMsg(String name) {
            this.name = name;
        }
    }
    
    public static Behavior<HelloMsg> create() {
        return Behaviors.setup(TestActor::new);
    }
    
    public TestActor(ActorContext<TestActor.HelloMsg> context) {
        super(context);
    }
    
    @Override
    public Receive<HelloMsg> createReceive() {
        return newReceiveBuilder().onMessage(HelloMsg.class, this::onHelloMsg).build();
    }
    
    private Behavior<HelloMsg> onHelloMsg(HelloMsg command) {
        System.out.println("hello from TestActor,"+command.name);
        return this;
    }
}

In [19]:
//#actor-system
final ActorSystem<TestActor.HelloMsg> testMain = ActorSystem.create(Behaviors.setup(TestActor::new), "test");
//#actor-system

//#main-send-messages
testMain.tell(new TestActor.HelloMsg("fff"));
//#main-send-messages

testMain.terminate();

04:31:48.323 [test-akka.actor.default-dispatcher-6] INFO akka.event.slf4j.Slf4jLogger - Slf4jLogger started
hello from TestActor,fff


In [20]:
public class GreeterMain extends AbstractBehavior<GreeterMain.SayHello> {

    public static class SayHello {
        public final String name;

        public SayHello(String name) {
            this.name = name;
        }
    }

    public static Behavior<SayHello> create() {
        return Behaviors.setup(GreeterMain::new);
    }

    private GreeterMain(ActorContext<SayHello> context) {
        super(context);
        //#create-actors
        //greeter = context.spawn(Greeter.create(), "greeter");
        //#create-actors
    }

    @Override
    public Receive<SayHello> createReceive() {
        return newReceiveBuilder().onMessage(SayHello.class, this::onSayHello).build();
    }

    private Behavior<SayHello> onSayHello(SayHello command) {
        ActorRef<TestActor.HelloMsg> testActor =
               getContext().spawn(TestActor.create(), "testActor");
        testActor.tell(new TestActor.HelloMsg("guci"));
        System.out.println("aaaaaa");
        return this;
    }
}


In [21]:
//#actor-system
final ActorSystem<GreeterMain.SayHello> greeterMain = ActorSystem.create(GreeterMain.create(), "helloakka");
//#actor-system

//#main-send-messages
greeterMain.tell(new GreeterMain.SayHello("Charles"));
//#main-send-messages

greeterMain.terminate();

04:32:03.109 [helloakka-akka.actor.default-dispatcher-3] INFO akka.event.slf4j.Slf4jLogger - Slf4jLogger started
hello from TestActor,guci
aaaaaa


# 异步编程

In [8]:
System.getProperty("java.version");

13.0.1

In [9]:
Runtime.version()

13.0.1+9-Ubuntu-216.04.1

In [22]:
CompletableFuture<String> future = CompletableFuture.supplyAsync(() -> {
//     try {
//         TimeUnit.SECONDS.sleep(1);
//     } catch (InterruptedException e) {
//         throw new IllegalStateException(e);
//     }
    System.out.println("I am working");
    return "Result of the asynchronous computation";
});
//String x=future.get();
//System.out.println(x);
//future.get()

I am working


In [11]:
CompletableFuture<Void> future = CompletableFuture.runAsync(() -> {
    // Simulate a long-running Job   
    try {
        TimeUnit.SECONDS.sleep(1);
    } catch (InterruptedException e) {
        throw new IllegalStateException(e);
    }
    System.out.println("I'll run in a separate thread than the main thread.");
});
future.get();

I'll run in a separate thread than the main thread.


In [12]:
import java.util.function.Supplier;
// Run a task specified by a Supplier object asynchronously
CompletableFuture<String> future = CompletableFuture.supplyAsync(new Supplier<String>() {
    @Override
    public String get() {
        try {
            TimeUnit.SECONDS.sleep(1);
        } catch (InterruptedException e) {
            throw new IllegalStateException(e);
        }
        return "Result of the asynchronous computation";
    }
});

// Block and get the result of the Future
String result = future.get();
System.out.println(result);


Result of the asynchronous computation


In [13]:
Executor executor = Executors.newFixedThreadPool(10);
CompletableFuture<String> future = CompletableFuture.supplyAsync(() -> {
    try {
        TimeUnit.SECONDS.sleep(1);
    } catch (InterruptedException e) {
        throw new IllegalStateException(e);
    }
    return "Result of the asynchronous computation";
}, executor);
future.get();

Result of the asynchronous computation

In [14]:
// Create a CompletableFuture
CompletableFuture<String> whatsYourNameFuture = CompletableFuture.supplyAsync(() -> {
   return "Rajeev";
});

// Attach a callback to the Future using thenApply()
CompletableFuture<String> greetingFuture = whatsYourNameFuture.thenApply(name -> {
   return "Hello " + name;
});

// Block and get the result of the future.
System.out.println(greetingFuture.get()); // Hello Rajeev



Hello Rajeev


In [15]:
import java.util.function.Function;
Function<String,String> f = (String s) -> {
    System.out.println("Executing111...");
    return s+" fffffff";
};

CompletableFuture<String> welcomeText = CompletableFuture.supplyAsync(() -> {
    return "Rajeev";
}).thenApply(name -> {
    System.out.println("ddddd");
    return "Hello " + name;
}).thenApply(greeting -> {
    return greeting + ", Welcome to the CalliCoder Blog";
})
.thenApply(f);

//System.out.println(welcomeText.get());



ddddd
Executing111...


In [16]:
CompletableFuture future=CompletableFuture.runAsync(()->{
    System.out.println("ddddd");
});
System.out.println("ppppp");

ddddd
ppppp


In [17]:
Executor executor = Executors.newFixedThreadPool(2);
CompletableFuture.supplyAsync(() -> {
    System.out.println("ddddd");
    return "Some result";
}).thenApplyAsync(result -> {
    // Executed in a thread obtained from the executor
    System.out.println("mmmm");
    return "Processed Result";
}, executor);


ddddd


java.util.concurrent.CompletableFuture@aa14204[Not completed]